In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

In [2]:
# load the starts data
starts_df = pd.read_csv('../capstone_data/skillshare_2022_starts.csv')

/Users/travisstowe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# clean the starts data.
starts_df = starts_df[starts_df['plan_length'] == 12]
starts_df = starts_df[starts_df['is_team'] == False]
starts_df = starts_df[starts_df['is_scholarship'] == False]
starts_df = starts_df[starts_df['is_direct_to_paid'] == False]
starts_df = starts_df[starts_df['had_trial'] == True]
starts_df = starts_df[starts_df['trial_length_offer'].isin(['One Week', 'One Month'])]

In [4]:
# all of these columns have the chance to be valuable to us.
valuable_cols = ['user_uid', 'is_active', 'create_time',
       'first_payment_time', 'last_payment_time', 'next_billing_time',
       'last_payment_attempt', 'last_failed_payment_attempt',
       'user_cancellation_time', 'cancellation_time', 'refund_time', 'coupon_id',
       'coupon_trial_length', 'payment_provider', 'payment_ux', 'is_paused',
       'pause_time', 'is_refunded', 'is_resume', 'paid_through', 'is_lapsed', 'is_pending', 'is_cancelled',
       'has_paid', 'is_paid', 'trial_end', 'category',
       'num_payments', 'num_successful_payments',
       'first_payment_currency_code', 'subscription_number',
       'paid_subscription_number', 'eligible_subscription_number',
       'is_p1_refunded', 'trial_extension_time',
       'original_create_time', 'original_trial_end', 'extended_trial_end',
       'was_trial_extended', 'is_trial_extension', 'is_split_trial',
       'is_paid_reactivation', 'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign',
       'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source',
       'eligible_trial_number']

starts_big_clean_df = starts_df[valuable_cols]

In [5]:
# these are the columns I bet will have some value to prediction.
prediction_cols = ['user_uid', 'create_time', 'first_payment_time', 'last_payment_attempt',
                   'last_failed_payment_attempt', 'user_cancellation_time', 'cancellation_time', 
                   'refund_time', 'coupon_id', 'coupon_trial_length', 'payment_provider', 'payment_ux', 
                   'is_refunded', 'has_paid', 'trial_end', 'first_payment_currency_code', 'original_trial_end', 
                   'extended_trial_end', 'was_trial_extended', 'is_trial_extension', 'is_split_trial', 
                   'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign', 
                   'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source', 'eligible_trial_number']

starts_clean_df = starts_big_clean_df[prediction_cols]

In [6]:
# make a day version of the trial start and end dates.
starts_clean_df['trial_end_day'] = pd.to_datetime(starts_clean_df.original_trial_end).dt.date
starts_clean_df['trial_start_day'] = pd.to_datetime(starts_clean_df.create_time).dt.date

# for some reason user_uid is a float only on this table. Change it to an int.
starts_clean_df['user_uid'] = starts_clean_df['user_uid'].astype(int)

In [7]:
# make a small df for merging onto the video views data.
trial_ends = starts_clean_df[['user_uid', 'trial_start_day', 'trial_end_day']]

In [8]:
file_ends = [x for x in range(63)]

In [9]:
full_user_df = pd.DataFrame()

for x in file_ends:
    print('working on file', x)
    filename = '../capstone_data/skillshare_2022_vviews_'+str(x)+'.csv'
    vviews_df = pd.read_csv(filename)
    # print('file', x, 'has', len(vviews_df), 'rows')
    vviews_df['view_date_dt'] = pd.to_datetime(vviews_df.view_date).dt.date
    vviews_df = vviews_df.merge(trial_ends, how='left', left_on='uid', right_on='user_uid')
    trial_vviews_df = vviews_df[vviews_df['view_date_dt'] < vviews_df['trial_end_day']]
    trial_vviews_df['day_of_trial'] = (trial_vviews_df['view_date_dt'] - trial_vviews_df['trial_start_day']).dt.days + 1
    trial_vviews_df = trial_vviews_df[trial_vviews_df['day_of_trial'] > 0]
    ttv_df = trial_vviews_df.groupby(by=['uid', 'day_of_trial']).agg(minutes_watched=pd.NamedAgg(column='sum', aggfunc='sum')).reset_index()
    vviews_pivot = ttv_df.pivot_table(
        index=['uid'],
        columns='day_of_trial',
        values='minutes_watched').reset_index().rename_axis(None, axis=1).reset_index(drop=True).fillna(0.0)
    #print('vviews_pivot for', x, 'has', len(vviews_pivot), 'rows')
    full_user_df = full_user_df.append(vviews_pivot, ignore_index=True)
    #print('full_user_df size:', len(full_user_df))
    

working on file 0
working on file 1
working on file 2
working on file 3
working on file 4
working on file 5
working on file 6
working on file 7
working on file 8
working on file 9
working on file 10
working on file 11
working on file 12
working on file 13
working on file 14
working on file 15
working on file 16
working on file 17
working on file 18
working on file 19
working on file 20
working on file 21
working on file 22
working on file 23
working on file 24
working on file 25
working on file 26
working on file 27
working on file 28
working on file 29
working on file 30
working on file 31
working on file 32
working on file 33
working on file 34
working on file 35
working on file 36
working on file 37
working on file 38
working on file 39
working on file 40
working on file 41
working on file 42
working on file 43
working on file 44
working on file 45
working on file 46
working on file 47
working on file 48
working on file 49
working on file 50
working on file 51
working on file 52
wor

In [10]:
for x in full_user_df.columns:
    if isinstance(x, int):
        colname = 'day-'+str(x)
        full_user_df.rename(columns={x: colname}, inplace=True)

In [11]:
print(len(full_user_df), len(full_user_df.uid.unique()))

388016 388016


In [12]:
full_user_df.to_csv('../capstone_data/skillshare_2022_all_views.csv')

In [15]:
clean_df = starts_df.copy()

In [16]:
clean_df['success'] = 0
clean_df['success'][clean_df['first_payment_time'].notnull()] = 1
clean_df['success'][clean_df['is_refunded']==1] = 0

In [17]:
clean_df['payment_provider_cat'] = clean_df['payment_provider'].astype('category')
clean_df['payment_provider_cat_codes'] = clean_df['payment_provider_cat'].cat.codes

clean_df['payment_ux_cat'] = clean_df['payment_ux'].astype('category')
clean_df['payment_ux_cat_codes'] = clean_df['payment_ux_cat'].cat.codes

clean_df['trial_length_offer_cat'] = clean_df['trial_length_offer'].astype('category')
clean_df['trial_length_offer_cat_codes'] = clean_df['trial_length_offer_cat'].cat.codes

clean_df['sub_utm_channel_cat'] = clean_df['sub_utm_channel'].astype('category')
clean_df['sub_utm_channel_cat_codes'] = clean_df['sub_utm_channel_cat'].cat.codes

clean_df['sub_utm_source_cat'] = clean_df['sub_utm_source'].astype('category')
clean_df['sub_utm_source_cat_codes'] = clean_df['sub_utm_source_cat'].cat.codes

clean_df['user_uid'] = clean_df['user_uid'].astype(int)